In [ ]:
import os
import tqdm as notebook_tqdm
from dotenv import load_dotenv

import dsp
import dspy
from dotdict import dotdict
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFinetune, BootstrapFewShotWithRandomSearch

load_dotenv()
unify_api_key = os.getenv("UNIFY_KEY")



In [ ]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=200, eval_seed=2023, dev_size=1000, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

In [ ]:
print(devset)
print(trainset)

In [ ]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

class QnA(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        prediction = self.generate_answer(question=question)
        return dspy.Prediction(answer=prediction.answer)

In [ ]:
gemma = dsp.Unify(
    endpoint="gemma-7b-it@anyscale",
    max_tokens=150,
    api_key=unify_api_key,
)

claude = dsp.Unify(
    endpoint="claude-3-haiku@anthropic",
    max_tokens=150,
    api_key=unify_api_key,
)

unify_router = dsp.Unify(
    endpoint="router@q:1|c:1.92e+00|t:7.45e-05|i:1.00e-06",
    max_tokens=150,
    api_key=unify_api_key,
)

turbo = dsp.Unify(
    endpoint="gpt-3.5-turbo@openai", 
    max_tokens=150,
    api_key=unify_api_key,

)

dspy.settings.configure(lm=turbo)

In [ ]:
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    return answer_EM

num_threads = 24

teleprompter = BootstrapFinetune(metric=validate_context_and_answer)



In [ ]:
compiled_qna = teleprompter.compile(QnA(), trainset=trainset[:50], valset=trainset[50:200])

In [ ]:
evaluate_hotpot = Evaluate(devset=devset[:1000], metric=validate_context_and_answer, num_threads=num_threads, display_progress=True, display_table=5)
evaluate_hotpot(compiled_qna, metric=validate_context_and_answer)

In [ ]:
claude.inspect_history(n=3)